In [1]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import warnings
import gc
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
%matplotlib inline

home_dir = "../data"

from matplotlib import rcParams

# figure size in inches
rcParams['figure.figsize'] = 20,10
rcParams['font.size'] = 22

In [2]:
csv_file = os.path.join(home_dir, "loan.csv")
pkl_file = os.path.join(home_dir, "loan.pkl")

if not os.path.exists(pkl_file):
    print("Reading csv file {}".format(csv_file))
    df = pd.read_csv(csv_file, low_memory=False)
    df.to_pickle(pkl_file)
    
else:    
    print("Reading pkl file {}".format(pkl_file))
    df = pd.read_pickle(pkl_file)
    
print("Done")
print("Shape {}".format(df.shape))

Reading csv file ../data\loan.csv
Done
Shape (2260668, 145)


In [3]:
# Use only those loans that are either Fully Paid or Charged Off

print(df.loan_status.value_counts())

fully_paid_df = df[df['loan_status'] == 'Fully Paid']
charged_off_df = df[df['loan_status'] == 'Charged Off']
local_df = fully_paid_df.append(charged_off_df)
print("Total loans either Fully Paid or charged off: {}".format(local_df.shape[0]))

print("Charged off percent: {0:.2f}%".format(100*charged_off_df.shape[0]/local_df.shape[0]))

Fully Paid                                             1041952
Current                                                 919695
Charged Off                                             261655
Late (31-120 days)                                       21897
In Grace Period                                           8952
Late (16-30 days)                                         3737
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     31
Name: loan_status, dtype: int64
Total loans either Fully Paid or charged off: 1303607
Charged off percent: 20.07%


In [4]:
# Refactor employment length and fill Null values - this might be an important indicator
local_df['emp_length'].fillna(value=0, inplace=True)
local_df['emp_length'].replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
local_df['emp_length'].replace(to_replace='', value=0, inplace=True)
local_df.loc[:,'emp_length'] = local_df.loc[:,'emp_length'].astype(int)

In [5]:
# Drop columns with 30% or more missing entries
percent_missing_threshold = 30

def null_values(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: 'Missing Values', 1: '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
    return mis_val_table_ren_columns

# Remove columns with too many missing values
miss_values = null_values(local_df)
cols_to_rm = miss_values[miss_values['% of Total Values'] > percent_missing_threshold].index
print("Removing columns: {}".format(cols_to_rm))

local_df.drop(cols_to_rm, axis=1, inplace=True)



Removing columns: Index(['id', 'next_pymnt_d', 'member_id', 'url',
       'orig_projected_additional_accrued_interest', 'hardship_type',
       'hardship_reason', 'hardship_status', 'hardship_last_payment_amount',
       'hardship_payoff_balance_amount', 'hardship_start_date',
       'hardship_loan_status', 'hardship_dpd', 'hardship_length',
       'payment_plan_start_date', 'hardship_end_date', 'deferral_term',
       'hardship_amount', 'sec_app_mths_since_last_major_derog',
       'sec_app_revol_util', 'revol_bal_joint',
       'sec_app_collections_12_mths_ex_med',
       'sec_app_chargeoff_within_12_mths', 'sec_app_num_rev_accts',
       'sec_app_open_act_il', 'sec_app_open_acc', 'sec_app_mort_acc',
       'sec_app_inq_last_6mths', 'sec_app_earliest_cr_line',
       'verification_status_joint', 'dti_joint', 'annual_inc_joint',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percentage', 'settlement_term', 'desc',
  

In [8]:
# remove null entries in dates
rows = local_df.shape[0]
#print(local_df.issue_d.isnull().sum(axis=0))
#print(local_df.last_pymnt_d.isnull().sum(axis=0))
#print(local_df.earliest_cr_line.isnull().sum(axis=0))

local_df = local_df.loc[~local_df.issue_d.isnull()]
local_df = local_df.loc[~local_df.last_pymnt_d.isnull()]
local_df = local_df.loc[~local_df.last_pymnt_d.isna()]
local_df = local_df.loc[~local_df.earliest_cr_line.isnull()]
local_df = local_df.loc[~local_df.last_credit_pull_d.isnull()]
print("Removed {0:.2f}% of rows because of null entries in date columns".format(100*(rows - local_df.shape[0])/rows))

print(local_df.shape)

Removed 0.18% of rows because of null entries in date columns
(1301293, 88)


In [9]:
# Transform dates to datetime format

local_df['issue_d'] = pd.to_datetime(local_df.issue_d, format='%b-%Y')
local_df['last_pymnt_d']  = pd.to_datetime(local_df.last_pymnt_d, format='%b-%Y')
local_df['earliest_cr_line']  = pd.to_datetime(local_df.earliest_cr_line, format='%b-%Y')
local_df['last_credit_pull_d'] = pd.to_datetime(local_df.last_credit_pull_d, format='%b-%Y')


# Number the months from earliest issue_d to latest last_pymnt_d
local_df['issue_month'] = local_df.issue_d.apply(lambda x: 12*x.year + x.month).astype(int)
local_df['last_p_month']  = local_df.last_pymnt_d.apply(lambda x: 12*x.year + x.month).astype(int)
local_df['earliest_cr_line_month']  = local_df.earliest_cr_line.apply(lambda x: 12*x.year + x.month).astype(int)
local_df['last_credit_pull_month']  = local_df.last_credit_pull_d.apply(lambda x: 12*x.year + x.month).astype(int)

local_df['earliest_cr_line_age'] = (local_df.issue_month - local_df.earliest_cr_line_month).apply(lambda x: max(x,0))

In [10]:
# Normalize so that earliest_issue_month is month 0

earliest_issue_month = min(local_df.issue_month)

local_df['issue_month']            -= earliest_issue_month
local_df['last_p_month']           -= earliest_issue_month
local_df['earliest_cr_line_month'] -= earliest_issue_month
local_df['last_credit_pull_month'] -= earliest_issue_month

# Create dictionaries to map between month numbers and dates
unique_i_months = sorted(local_df.issue_month.unique())
unique_p_months = sorted(local_df.last_p_month.unique())

months = set(unique_i_months + unique_p_months)

unique_i_d = sorted(pd.to_datetime(local_df.issue_d.unique()))
unique_p_d = sorted(pd.to_datetime(local_df.last_pymnt_d.unique()))

dates = set(unique_i_d + unique_p_d)

month_to_date = dict(zip(months, dates))
date_to_month = dict(zip(dates, months))

# Print out some date stats
earliest_issue_month = min(unique_i_months)
latest_issue_month = max(unique_i_months) 
earliest_last_payment_month = min(unique_p_months)
latest_last_payment_month     = max(unique_p_months)

earliest_issue_d = min(unique_i_d)
latest_issue_d = max(unique_i_d) 
earliest_last_payment_d = min(unique_p_d)
latest_last_payment_d     = max(unique_p_d)

print("Issue months from: {} to: {}".format(earliest_issue_month, latest_issue_month))
print("Last payment months from: {} to: {}".format(earliest_last_payment_month, latest_last_payment_month))

print("Issue dates from: {} to: {}".format(earliest_issue_d.strftime("%b-%Y"), latest_issue_d.strftime("%b-%Y")))
print("Last payment dates from: {} to: {}".format(earliest_last_payment_d.strftime("%b-%Y"), latest_last_payment_d.strftime("%b-%Y")))

Issue months from: 0 to: 138
Last payment months from: 7 to: 140
Issue dates from: Jun-2007 to: Dec-2018
Last payment dates from: Jan-2008 to: Feb-2019


In [11]:
# Calculate analytical quantities
analytics = ['TotalGain','PvGain' ,'LoanStatus', 'LifeOfLoan']

# Total Gain
#   principal received minus funded amount
#   plus interest
#   plus recoveries in case of default
#   minus collection recovery fee
local_df['TotalGain'] = local_df['total_rec_prncp'] - local_df['funded_amnt'] \
                        + local_df['total_rec_int'] + local_df['recoveries'] - local_df['collection_recovery_fee']

# loan_status: 
#    ChargedOff = 1
#    Fully Paid = 0
local_df['LoanStatus'] = (local_df['loan_status']!='Fully Paid').astype(int)


# Life of loan
#   begins on issue month
#   ends on last payment month
#   ... so if loan issued on month 0 and ends on month 1, then there is one payment recieved.
local_df['LifeOfLoan'] = local_df.last_p_month - local_df.issue_month


# Present Value - Assume equal monthly payments totalling total gain for present value calculation

ann_rate = 2.0

mo_rate = ann_rate/12

def pv(q, rate):
    discount = 1/(1+rate)
    pv = 0
    for value in reversed(q):
        pv *= discount
        #print(pv)
        pv += value
        #print(pv)
    return pv


def calc_pv(row):
    if row['LifeOfLoan'] == 0:
        return row['TotalGain']
    eq_pymnt = row['TotalGain'] / row['LifeOfLoan']
    q = [eq_pymnt]*int(row['LifeOfLoan'])
    return pv(q, mo_rate)

local_df['PvGain'] = local_df.apply(calc_pv, axis=1)

local_df[[ 'loan_amnt','funded_amnt','TotalGain','PvGain']].head()

,loan_amnt,funded_amnt,TotalGain,PvGain
100,30000,30000,26.44,26.440000
152,40000,40000,856.68,795.488571
170,20000,20000,215.79,200.376429
186,4500,4500,49.22,45.704286
215,8425,8425,310.15,287.996429


# Process Features

In [12]:
threshold_df = local_df
print(analytics)
print(sorted(threshold_df.columns))

['TotalGain', 'PvGain', 'LoanStatus', 'LifeOfLoan']
['LifeOfLoan', 'LoanStatus', 'PvGain', 'TotalGain', 'acc_now_delinq', 'acc_open_past_24mths', 'addr_state', 'annual_inc', 'application_type', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'debt_settlement_flag', 'delinq_2yrs', 'delinq_amnt', 'disbursement_method', 'dti', 'earliest_cr_line', 'earliest_cr_line_age', 'earliest_cr_line_month', 'emp_length', 'emp_title', 'funded_amnt', 'funded_amnt_inv', 'grade', 'hardship_flag', 'home_ownership', 'initial_list_status', 'inq_last_6mths', 'installment', 'int_rate', 'issue_d', 'issue_month', 'last_credit_pull_d', 'last_credit_pull_month', 'last_p_month', 'last_pymnt_amnt', 'last_pymnt_d', 'loan_amnt', 'loan_status', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num

### Encode object columns

In [13]:
# Feature segmentations

datetimetypes = ['issue_d', 'last_pymnt_d', 'earliest_cr_line', 'last_credit_pull_d' ]

date_identifiers = ['issue_month', 'last_p_month','earliest_cr_line_month', 'last_credit_pull_month',
                   'int_rate', 'installment']

redundants = ['grade']

not_possible_to_know_at_decision_time = [
    'funded_amnt', 'funded_amnt_inv',
    'collection_recovery_fee', 'debt_settlement_flag', 
    'hardship_flag', 
    'last_pymnt_amnt', 'loan_status', 
    'out_prncp', 'out_prncp_inv',
    'pymnt_plan', 'recoveries', 'total_pymnt', 'total_pymnt_inv', 
    'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp'
]

too_many = ['emp_title', 'title', 'zip_code']

exclude_features = datetimetypes + date_identifiers + redundants + not_possible_to_know_at_decision_time + too_many

features = [col for col in threshold_df.columns if col not in exclude_features]

In [14]:
threshold_df[features].select_dtypes(['object']).apply(pd.Series.nunique, axis = 0)

term                    2
sub_grade              35
home_ownership          6
verification_status     3
purpose                14
addr_state             51
initial_list_status     2
application_type        2
disbursement_method     2
dtype: int64

In [15]:
encoded_df = threshold_df[features].copy()

exclude_df = threshold_df[exclude_features].copy()


In [16]:
# label encode the binary categories
from sklearn import preprocessing
count = 0

for col in features:
    try:
        if encoded_df[col].dtype == 'object':
            if len(list(encoded_df[col].unique())) <= 2:     
                le = preprocessing.LabelEncoder()
                encoded_df[col] = le.fit_transform(encoded_df[col])
                count += 1
                print (col)
    except:
        print (col)
            
print('%d columns were label encoded.' % count)

# One-hot encode the rest of the object columns
encoded_df = pd.get_dummies(encoded_df, sparse=True)


term
initial_list_status
application_type
disbursement_method
4 columns were label encoded.


In [17]:
# Use SimpleImputer to handle bad entries 
from sklearn.impute import SimpleImputer

simple_imputer = SimpleImputer(strategy='mean')

imputer = simple_imputer.fit(encoded_df)
imputed = imputer.transform(encoded_df)

encoded_df = pd.DataFrame(imputed, columns=encoded_df.columns, index=exclude_df.index)

In [18]:
# Glue the encoded_df and exclude_df back together

data_df = pd.concat([encoded_df, exclude_df], axis=1, sort=False)


In [19]:
import pickle

processed_data_pkl = os.path.join(home_dir, "processed_data.pkl")

processed_data = (data_df, exclude_features, analytics)

with open(processed_data_pkl, 'wb') as f:
    pickle.dump(processed_data, f)